In [1]:
import crawl4ai
print(crawl4ai.__version__.__version__)

0.6.3


In [2]:
!crawl4ai-doctor

[INIT].... → Running Crawl4AI health check... 
[INIT].... → Crawl4AI 0.6.3 
[TEST].... ℹ Testing crawling capabilities... 
[EXPORT].. ℹ Exporting media (PDF/MHTML/screenshot) took 1.72s 
[FETCH]... ↓ https://crawl4ai.com                                               
| ✓ | ⏱: 7.28s 
[SCRAPE].. ◆ https://crawl4ai.com                                               
| ✓ | ⏱: 0.12s 
[COMPLETE] ● https://crawl4ai.com                                               
| ✓ | ⏱: 7.41s 
[COMPLETE] ● ✅ Crawling test passed! 


# try the code below to ensure that Playwright is installed and works properly.

In [3]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [4]:
from playwright.async_api import async_playwright

async def test_browser():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto('https://example.com')
        print(f'Title: {await page.title()}')
        await browser.close()

asyncio.run(test_browser())

Title: Example Domain


# simple crawling

In [8]:
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode

async def simple_crawl():
    crawler_run_config = CrawlerRunConfig(cache_mode=CacheMode.ENABLED) # CacheMode.BYPASS means no cache will be used.
    async with AsyncWebCrawler() as crawler:
        result = await crawler.arun(
            url="https://mushfiqur-rahman-robin.github.io/",
            config=crawler_run_config
        )
        print(result.markdown.raw_markdown[:500].replace("\n", " -- "))  # Print the first 500 characters

asyncio.run(simple_crawl())

[INIT].... → Crawl4AI 0.6.3 

[FETCH]... ↓ https://mushfiqur-rahman-robin.github.io/                                                            |
✓ | ⏱: 1.20s 

[SCRAPE].. ◆ https://mushfiqur-rahman-robin.github.io/                                                            |
✓ | ⏱: 0.04s 

[COMPLETE] ● https://mushfiqur-rahman-robin.github.io/                                                            |
✓ | ⏱: 1.29s 

[ Md. Mushfiqur Rahman ](https://Mushfiqur-Rahman-Robin.github.io/) --   * [ About Me ](https://Mushfiqur-Rahman-Robin.github.io/about/ "About Me page") --   * [ Achievements ](https://Mushfiqur-Rahman-Robin.github.io/achievements/ "Achievements page") --   * [ Certifications ](https://Mushfiqur-Rahman-Robin.github.io/certifications/ "Certifications page") --   * [ Contact Me ](https://Mushfiqur-Rahman-Robin.github.io/contact/ "Contact Me page") --   * [ Projects ](https://Mushfiqur-Rahman-Robin.github.io/post


# dynamic content crawling

In [12]:
async def crawl_dynamic_content():
    # You can use wait_for to wait for a condition to be met before returning the result
    # wait_for = """() => {
    #     return Array.from(document.querySelectorAll('article.tease-card')).length > 10;
    # }"""

    # wait_for can be also just a css selector
    # wait_for = "article.tease-card:nth-child(10)"

    async with AsyncWebCrawler() as crawler:
        js_code = [
            """
            var btns = document.querySelectorAll('button');
            for (var i = 0; i < btns.length; i++) {
                if (btns[i].textContent.includes('Load More')) {
                    btns[i].click();
                    break;
                }
            }
            """
        ]
        # the js_code parameter lets you run custom JavaScript code in the context of the web page before scraping. 
        # This is especially useful for interacting with dynamic web content — things like clicking buttons, filling forms, or waiting for data to load via JavaScript.
        config = CrawlerRunConfig(
            cache_mode=CacheMode.ENABLED,
            js_code=js_code,
            # wait_for=wait_for,
        )
        result = await crawler.arun(
            url="https://www.nbcnews.com/business",
            config=config,

        )
        print(result.markdown.raw_markdown[:500].replace("\n", " -- "))  # Print first 500 characters

asyncio.run(crawl_dynamic_content())

[INIT].... → Crawl4AI 0.6.3 

[FETCH]... ↓ https://www.nbcnews.com/business                                                                     |
✓ | ⏱: 0.02s 

[COMPLETE] ● https://www.nbcnews.com/business                                                                     |
✓ | ⏱: 0.02s 

IE 11 is not supported. For an optimal experience visit our site on another browser. -- Skip to Content -- [NBC News Logo](https://www.nbcnews.com) -- Sponsored By --   * [ Politics](https://www.nbcnews.com/politics) --   * Local --   * [New York](https://www.nbcnews.com/new-york) --   * [Los Angeles](https://www.nbcnews.com/los-angeles) --   * [Chicago](https://www.nbcnews.com/chicago) --   * [Dallas-Fort Worth](https://www.nbcnews.com/dallas-fort-worth) --   * [Philadelphia](https://www.nbcnews.com/philadelphia) --   * [Washi


# content cleaning

In [14]:
from crawl4ai.content_filter_strategy import PruningContentFilter
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator

async def clean_content():
    async with AsyncWebCrawler(verbose=True) as crawler:
        config = CrawlerRunConfig(
            cache_mode=CacheMode.ENABLED,
            excluded_tags=['nav', 'footer', 'aside'],
            remove_overlay_elements=True,
            # excluded_tags: Removes HTML sections like <nav>, <footer>, and <aside> (often boilerplate or repeated UI).
            # remove_overlay_elements=True: Automatically removes modal overlays/popups, like cookie consent banners.
            markdown_generator=DefaultMarkdownGenerator(
                content_filter=PruningContentFilter(threshold=0.48, threshold_type="fixed", min_word_threshold=0),
                options={
                    "ignore_links": True
                }
            ),
        )
        result = await crawler.arun(
            url="https://en.wikipedia.org/wiki/Apple",
            config=config,
        )
        full_markdown_length = len(result.markdown.raw_markdown)
        fit_markdown_length = len(result.markdown.fit_markdown)
        print(f"Full Markdown Length: {full_markdown_length}")
        print(f"Fit Markdown Length: {fit_markdown_length}")

        with open("output/apple_markdown.md", "w") as f:
            f.write(result.markdown.fit_markdown)

# This code:
# Visits a webpage (Wikipedia on Apple 🍎)
# Cleans it (removes UI elements and overlays)
# Converts it into clean, minimal markdown
# Filters low-value content (using a scoring mechanism)
# Returns both the full and filtered markdown lengths


asyncio.run(clean_content())

[INIT].... → Crawl4AI 0.6.3 

[FETCH]... ↓ https://en.wikipedia.org/wiki/Apple                                                                  |
✓ | ⏱: 0.02s 

[COMPLETE] ● https://en.wikipedia.org/wiki/Apple                                                                  |
✓ | ⏱: 0.02s 

Full Markdown Length: 92190
Fit Markdown Length: 0


# link analysis

In [21]:
async def link_analysis():
    async with AsyncWebCrawler() as crawler:
        config = CrawlerRunConfig(
            cache_mode=CacheMode.ENABLED,
            exclude_external_links=False,  # If you want to collect & save them
            exclude_social_media_links=True,
            exclude_domains=["facebook.com", "twitter.com"]
        )
        result = await crawler.arun(
            url="https://www.nbcnews.com/business",
            config=config,
        )

        internal_links = result.links.get('internal', [])
        external_links = result.links.get('external', [])

        print(f"Found {len(internal_links)} internal links")
        print(f"Found {len(external_links)} external links")

        with open("output/link_analysis_markdown.md", "w") as f:
            f.write("# Internal Links\n")
            for link in internal_links:
                f.write(f"- [{link['text']}]({link['href']})\n")

            f.write("\n# External Links\n")
            for link in external_links:
                f.write(f"- [{link['text']}]({link['href']})\n")

        for link in internal_links[:5]:
            print(f"Href: {link['href']}\nText: {link['text']}\n")

        for link in external_links[:5]:
            print(f"External Href: {link['href']}\nText: {link['text']}\n")


asyncio.run(link_analysis())

[INIT].... → Crawl4AI 0.6.3 

[FETCH]... ↓ https://www.nbcnews.com/business                                                                     |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://www.nbcnews.com/business                                                                     |
✓ | ⏱: 0.02s 

Found 109 internal links
Found 56 external links
Href: https://www.nbcnews.com
Text: NBC News Logo

Href: https://www.nbcnews.com/politics
Text: Politics

Href: https://www.nbcnews.com/new-york
Text: New York

Href: https://www.nbcnews.com/los-angeles
Text: Los Angeles

Href: https://www.nbcnews.com/chicago
Text: Chicago

External Href: https://www.facebook.com/sharer/sharer.php?u=null&cid=article_share_facebook
Text: 

External Href: https://x.com/intent/post?text=&via=nbcnews&url=null&original_referer=URL&cid=article_share_twitter
Text: 

External Href: mailto:?subject=&body=null
Text: 

External Href: https://www.today.com/
Text: Today

External Href: https://www.msnbc.com/
Text: MSNBC



# media handling

In [40]:
import os
async def media_handling():
    os.makedirs("output/media", exist_ok=True)

    async with AsyncWebCrawler() as crawler:
        config = CrawlerRunConfig(
            cache_mode=CacheMode.ENABLED,
            exclude_external_images=False,
            screenshot=False,  # No screenshot needed
            image_score_threshold=0.5,
            exclude_internal_links=False
        )
        result = await crawler.arun(
            url="https://mushfiqur-rahman-robin.github.io/publications/",
            config=config,
        )

        for img in result.media['images'][:5]:
            print(f"Image URL: {img['src']}, Alt: {img['alt']}, Score: {img['score']}")

        markdown_lines = ["# 🖼️ All Images from the Website\n"]

        all_images = result.media.get('images', [])

        if not all_images:
            markdown_lines.append("_No images found on the page._")
        else:
            for i, img in enumerate(all_images):
                image_url = img['src']
                alt_text = img.get('alt', 'No alt text')
                score = img.get('score', 0)

                markdown_lines.append(f"### Image {i+1}")
                markdown_lines.append(f"- **Alt**: {alt_text}")
                markdown_lines.append(f"- **Score**: {score}")
                markdown_lines.append(f"- **Link**: [{image_url}]({image_url})")
                markdown_lines.append(f"![Image {i+1}]({image_url})\n")

        with open("output/media_summary.md", "w") as md_file:
            md_file.write("\n".join(markdown_lines))

asyncio.run(media_handling())

[INIT].... → Crawl4AI 0.6.3 

[FETCH]... ↓ https://mushfiqur-rahman-robin.github.io/publications/                                               |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://mushfiqur-rahman-robin.github.io/publications/                                               |
✓ | ⏱: 0.01s 

Image URL: https://Mushfiqur-Rahman-Robin.github.io/images/BIMConference.PNG, Alt: , Score: 2
Image URL: https://Mushfiqur-Rahman-Robin.github.io/images/EICT_presentation_certificate.PNG, Alt: , Score: 1


In [36]:
async def media_handling():
    async with AsyncWebCrawler() as crawler:
        config = CrawlerRunConfig(
            cache_mode=CacheMode.ENABLED,
            exclude_external_images=False,
            # screenshot=True # Set this to True if you want to take a screenshot
        )
        result = await crawler.arun(
            url="https://www.nbcnews.com/business",
            config=config,
        )
        for img in result.media['images'][:5]:
            print(f"Image URL: {img['src']}, Alt: {img['alt']}, Score: {img['score']}")

asyncio.run(media_handling())

[INIT].... → Crawl4AI 0.6.3 

[FETCH]... ↓ https://www.nbcnews.com/business                                                                     |
✓ | ⏱: 0.14s 

[COMPLETE] ● https://www.nbcnews.com/business                                                                     |
✓ | ⏱: 0.15s 

Image URL: https://media-cldnry.s-nbcnews.com/image/upload/t_focal-762x508,f_auto,q_auto:best/rockcms/2025-06/250602-jamie-dimon-se-517p-f32603.jpg, Alt: Jamie Dimon., Score: 6
Image URL: https://media-cldnry.s-nbcnews.com/image/upload/t_focal-762x508,f_auto,q_auto:best/rockcms/2025-05/250514-uber-ch-1315-82ca9e.jpg, Alt: Uber headquarters., Score: 6
Image URL: https://media-cldnry.s-nbcnews.com/image/upload/t_focal-80x80,f_auto,q_auto:best/rockcms/2025-05/250516-Musk-RS-cbcca6.jpg, Alt: Elon Musk, Score: 6
Image URL: https://media-cldnry.s-nbcnews.com/image/upload/t_focal-80x80,f_auto,q_auto:best/rockcms/2025-06/250602-empty-office-stock-se-505p-55a14f.jpg, Alt: Empty table and chair against window at new workplace, Score: 6
Image URL: https://media-cldnry.s-nbcnews.com/image/upload/t_focal-80x80,f_auto,q_auto:best/rockcms/2025-06/250602-Byron-Allen-2019-ac-510p-b3ed94.jpg, Alt: byron allen portrait, Score: 6


# Using Hooks for Custom Workflow

In [42]:
from playwright.async_api import Page, BrowserContext

In [45]:
async def before_goto(page: Page, context: BrowserContext, url: str, **kwargs):
        """Hook called before navigating to each URL"""
        print(f"[HOOK] before_goto - About to visit: {url}")
        # Example: Add custom headers for the request
        await page.set_extra_http_headers({
            "Custom-Header": "my-value"
        })
        return page

async def custom_hook_workflow(verbose=True):
    async with AsyncWebCrawler(config=BrowserConfig( verbose=verbose)) as crawler:
        # Set a 'before_goto' hook to run custom code just before navigation
        crawler.crawler_strategy.set_hook("before_goto", before_goto)

        # Perform the crawl operation
        result = await crawler.arun(
            url="https://crawl4ai.com",
            config=CrawlerRunConfig(cache_mode=CacheMode.BYPASS)
        )
        print(result.markdown.raw_markdown[:500].replace("\n", " -- "))
        with open("output/custom_hook_output.md", "w") as f:
            f.write(result.markdown.raw_markdown)

asyncio.run(custom_hook_workflow())

[INIT].... → Crawl4AI 0.6.3 

[HOOK] before_goto - About to visit: https://crawl4ai.com


[FETCH]... ↓ https://crawl4ai.com                                                                                 |
✓ | ⏱: 5.36s 

[SCRAPE].. ◆ https://crawl4ai.com                                                                                 |
✓ | ⏱: 0.12s 

[COMPLETE] ● https://crawl4ai.com                                                                                 |
✓ | ⏱: 5.49s 

[Crawl4AI Documentation (v0.6.x)](https://docs.crawl4ai.com/) --   * [ Home ](https://docs.crawl4ai.com/) --   * [ Ask AI ](https://docs.crawl4ai.com/core/ask-ai/) --   * [ LLM Context ](https://docs.crawl4ai.com/core/llmtxt/) --   * [ Quick Start ](https://docs.crawl4ai.com/core/quickstart/) --   * [ Code Examples ](https://docs.crawl4ai.com/core/examples/) --   * [ Search ](https://docs.crawl4ai.com/) --  --  -- × --   * Home --   * [Ask AI](https://docs.crawl4ai.com/core/ask-ai/) --   * [LLM Context](https://docs.crawl4ai.com/c


# list of some available webhooks

List of available hooks and examples for each stage of the crawling process:

- **on_browser_created**
    ```python
    async def on_browser_created_hook(browser):
        print("[Hook] Browser created")
    ```

- **before_goto**
    ```python
    async def before_goto_hook(page, context = None):
        await page.set_extra_http_headers({"X-Test-Header": "test"})
    ```

- **after_goto**
    ```python
    async def after_goto_hook(page, context = None):
        print(f"[Hook] Navigated to {page.url}")
    ```

- **on_execution_started**
    ```python
    async def on_execution_started_hook(page, context = None):
        print("[Hook] JavaScript execution started")
    ```

- **before_return_html**
    ```python
    async def before_return_html_hook(page, html, context = None):
        print(f"[Hook] HTML length: {len(html)}")
    ```

# session-based crawling

session-based crawling is especially beneficial when navigating through multi-page content where each page load needs to maintain the same session context.

In [ ]:
from crawl4ai.extraction_strategy import (
    JsonCssExtractionStrategy,
    LLMExtractionStrategy,
)
import json

async def crawl_dynamic_content_pages_method_2():
    print("\n--- Advanced Multi-Page Crawling with JavaScript Execution ---")

    async with AsyncWebCrawler() as crawler:
        url = "https://github.com/microsoft/TypeScript/commits/main"
        session_id = "typescript_commits_session"
        all_commits = []

        js_next_page_and_wait = """
        (async () => {
            const getCurrentCommit = () => {
                const commits = document.querySelectorAll('li.Box-sc-g0xbh4-0 h4');
                return commits.length > 0 ? commits[0].textContent.trim() : null;
            };

            const initialCommit = getCurrentCommit();
            const button = document.querySelector('a[data-testid="pagination-next-button"]');
            if (button) button.click();

            // Poll for changes
            while (true) {
                await new Promise(resolve => setTimeout(resolve, 100)); // Wait 100ms
                const newCommit = getCurrentCommit();
                if (newCommit && newCommit !== initialCommit) {
                    break;
                }
            }
        })();
        """

        schema = {
            "name": "Commit Extractor",
            "baseSelector": "li.Box-sc-g0xbh4-0",
            "fields": [
                {
                    "name": "title",
                    "selector": "h4.markdown-title",
                    "type": "text",
                    "transform": "strip",
                },
            ],
        }
        extraction_strategy = JsonCssExtractionStrategy(schema, verbose=True)

        for page in range(2):  # Crawl 2 pages
            config = CrawlerRunConfig(
                cache_mode=CacheMode.BYPASS,
                session_id=session_id,
                css_selector="li.Box-sc-g0xbh4-0",
                extraction_strategy=extraction_strategy,
                js_code=js_next_page_and_wait if page > 0 else None,
                js_only=page > 0,
            )
            result = await crawler.arun(
                url=url,
                config=config
            )

            assert result.success, f"Failed to crawl page {page + 1}"

            commits = json.loads(result.extracted_content)
            all_commits.extend(commits)

            print(f"Page {page + 1}: Found {len(commits)} commits")

        await crawler.crawler_strategy.kill_session(session_id)
        print(f"Successfully crawled {len(all_commits)} commits across 3 pages")

asyncio.run(crawl_dynamic_content_pages_method_2())


--- Advanced Multi-Page Crawling with JavaScript Execution ---


[INIT].... → Crawl4AI 0.6.3 

[FETCH]... ↓ https://github.com/microsoft/TypeScript/commits/main                                                 |
✓ | ⏱: 3.60s 

[SCRAPE].. ◆ https://github.com/microsoft/TypeScript/commits/main                                                 |
✓ | ⏱: 0.00s 

[EXTRACT]. ■ Completed for https://github.com/microsoft/TypeScript/commits/ma... | Time: 0.0009419260022696108s 

...

[EXTRACT]. ■ Completed for https://github.com/microsoft/TypeScript/commits/ma... | Time: 0.0024015809904085472s 

[COMPLETE] ● https://github.com/microsoft/TypeScript/commits/main                                                 |
✓ | ⏱: 0.68s 

Page 2: Found 0 commits


[WARNING]. ⚠ Session auto-kill is enabled in the new version. No need to manually kill sessions. 

Successfully crawled 0 commits across 3 pages


# Executing JavaScript & Extract Structured Data without LLMs

In [53]:
async def extract():
    schema = {
        "name": "KidoCode Courses",
        "baseSelector": "section.charge-methodology .div-block-214.p-extraxx",
        "fields": [
            {
                "name": "section_title",
                "selector": "h3.heading-50",
                "type": "text",
            },
            {
                "name": "section_description",
                "selector": ".charge-content",
                "type": "text",
            },
            {
                "name": "course_name",
                "selector": ".text-block-93",
                "type": "text",
            },
            {
                "name": "course_description",
                "selector": ".course-content-text",
                "type": "text",
            },
            {
                "name": "course_icon",
                "selector": ".image-92",
                "type": "attribute",
                "attribute": "src"
            }
        ]
    }

    extraction_strategy = JsonCssExtractionStrategy(schema, verbose=True)

    async with AsyncWebCrawler() as crawler:

        # Create the JavaScript that handles clicking multiple times
        js_click_tabs = """
        (async () => {
            const tabs = document.querySelectorAll("section.charge-methodology .tabs-menu-3 > div");

            for(let tab of tabs) {
                // scroll to the tab
                tab.scrollIntoView();
                tab.click();
                // Wait for content to load and animations to complete
                await new Promise(r => setTimeout(r, 500));
            }
        })();
        """

        config = CrawlerRunConfig(
            cache_mode=CacheMode.BYPASS,
            extraction_strategy=extraction_strategy,
            js_code=[js_click_tabs],
        )
        result = await crawler.arun(
            url="https://www.kidocode.com/degrees/technology",
            config=config
        )

        courses = json.loads(result.extracted_content)
        print(result.extracted_content)
        print(f"Successfully extracted {len(courses)} courses")
        print(len(result.markdown))
        # print(json.dumps(courses[0], indent=2))

        with open("output/kidocode_courses.json", "w") as f:
            json.dump(courses, f, indent=2)

await extract()

[INIT].... → Crawl4AI 0.6.3 

[FETCH]... ↓ https://www.kidocode.com/degrees/technology                                                          |
✓ | ⏱: 3.63s 

[SCRAPE].. ◆ https://www.kidocode.com/degrees/technology                                                          |
✓ | ⏱: 0.20s 

[EXTRACT]. ■ Completed for https://www.kidocode.com/degrees/technology... | Time: 0.1189301760023227s 

[COMPLETE] ● https://www.kidocode.com/degrees/technology                                                          |
✓ | ⏱: 3.97s 

[
    {
        "section_title": "Coding",
        "section_description": "Introducing students to the world of programming through Python, emphasizing foundational coding concepts and creativity in solving problems through generative art and geometric patterns.",
        "course_name": "Coding with Python",
        "course_description": "Explore the exciting intersection of art and technology in Creative Coding with Python. This engaging course introduces young learners to Python programming, teaching them foundational coding concepts through the creation of generative art and geometric patterns. As students progress from basic programming principles to artistic project development, they'll enhance their problem-solving and logical thinking abilities, all within a creative and fun learning environment.",
        "course_icon": "https://cdn.prod.website-files.com/61d6943d6b59241863c825d6/65c0a4d516aec7d103edb1e5_Python-Data-Science-Toolkit-Icon.svg"
    },
    {
        "section_title"

# LLM-based extraction

In [ ]:
from pydantic import BaseModel, Field
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

class OpenAIModelFee(BaseModel):
    model_name: str = Field(..., description="Name of the OpenAI model.")
    input_fee: str = Field(..., description="Fee for input token for the OpenAI model.")
    output_fee: str = Field(
        ..., description="Fee for output token for the OpenAI model."
    )

async def extract_structured_data_using_llm(provider: str, api_token: str = None, extra_headers: dict = None):
    print(f"\n--- Extracting Structured Data with {provider} ---")

    # Skip if API token is missing (for providers that require it)
    if api_token is None and provider != "ollama":
        print(f"API token is required for {provider}. Skipping this example.")
        return

    extra_args = {"extra_headers": extra_headers} if extra_headers else {}

    async with AsyncWebCrawler(verbose=True) as crawler:
        result = await crawler.arun(
            url="https://openai.com/api/pricing/",
            word_count_threshold=1,
            extraction_strategy=LLMExtractionStrategy(
                provider=provider,
                api_token=api_token,
                schema=OpenAIModelFee.schema(),
                extraction_type="schema",
                instruction="""Extract all model names along with fees for input and output tokens."
                "{model_name: 'GPT-4', input_fee: 'US$10.00 / 1M tokens', output_fee: 'US$30.00 / 1M tokens'}.""",
                **extra_args
            ),
            cache_mode = CacheMode.ENABLED
        )
        print(json.loads(result.extracted_content)[:5])

# Usage:
# await extract_structured_data_using_llm("huggingface/meta-llama/Meta-Llama-3.1-8B-Instruct", os.getenv("HUGGINGFACE_API_KEY"))
# await extract_structured_data_using_llm("ollama/llama3.2")
await extract_structured_data_using_llm("openai/gpt-4o-mini", os.getenv("OPENAI_API_KEY"))

# cosine similary approach

In [ ]:
from crawl4ai.extraction_strategy import CosineStrategy

async def cosine_similarity_extraction():
    async with AsyncWebCrawler() as crawler:
        strategy = CosineStrategy(
            word_count_threshold=10,
            max_dist=0.2, # Maximum distance between two words
            linkage_method="ward", # Linkage method for hierarchical clustering (ward, complete, average, single)
            top_k=3, # Number of top keywords to extract
            sim_threshold=0.3, # Similarity threshold for clustering
            semantic_filter="McDonald's economic impact, American consumer trends", # Keywords to filter the content semantically using embeddings
            verbose=True
        )

        result = await crawler.arun(
            url="https://www.nbcnews.com/business/consumer/how-mcdonalds-e-coli-crisis-inflation-politics-reflect-american-story-rcna177156",
            extraction_strategy=strategy,
            cache_mode = CacheMode.ENABLED
        )
        print(json.loads(result.extracted_content)[:5])
        with open("output/cosine_similarity_extraction.json", "w") as f:
            json.dump(json.loads(result.extracted_content), f, indent=2)

asyncio.run(cosine_similarity_extraction())